## Load results for each method

In [151]:
import pandas as pd

df_cot = pd.read_csv("results/cot_few-shot/cot_few-shot_results_2025-03-27_13-55-58.csv")
df_cove = pd.read_csv("results/cove_few-shot/cove_few-shot_results_2025-03-27_14-42-13.csv")
df_self_refine = pd.read_csv("results/self-refine_few-shot/self-refine_few-shot_results_2025-03-27_15-08-17.csv")

### Rename columns before merging

In [152]:
df_cot = df_cot.rename(columns={"predicted": "predicted_cot"})
df_cove = df_cove.rename(columns={"predicted": "predicted_cove"})
df_self_refine = df_self_refine.rename(columns={"predicted": "predicted_sr"})

### Merge results on question ID

In [153]:
df_all = df_cot[["id", "subject", "ground_truth", "predicted_cot"]].merge(
    df_cove[["id", "predicted_cove"]],
    on="id"
).merge(
    df_self_refine[["id", "predicted_sr"]],
    on="id"
)

df_all.head()

,id,subject,ground_truth,predicted_cot,predicted_cove,predicted_sr
0,questao_01,"Linguagens, Códigos e suas Tecnologias",A,A,A,A
1,questao_02,"Linguagens, Códigos e suas Tecnologias",A,D,D,D
2,questao_03,"Linguagens, Códigos e suas Tecnologias",C,C,C,C
3,questao_04,"Linguagens, Códigos e suas Tecnologias",E,E,D,E
4,questao_05,"Linguagens, Códigos e suas Tecnologias",A,A,A,A


## Calculate Accuracy

### Add correctness flags for each method

In [154]:
df_all["correct_cot"] = df_all["predicted_cot"] == df_all["ground_truth"]
df_all["correct_cove"] = df_all["predicted_cove"] == df_all["ground_truth"]
df_all["correct_sr"] = df_all["predicted_sr"] == df_all["ground_truth"]

### Calculate general accuracy

In [155]:
acc_cot = df_all["correct_cot"].mean()
acc_cove = df_all["correct_cove"].mean()
acc_sr = df_all["correct_sr"].mean()

print(f"Accuracy – CoT:          {acc_cot:.2%}")
print(f"Accuracy – CoVe:         {acc_cove:.2%}")
print(f"Accuracy – Self-Refine:  {acc_sr:.2%}")

Accuracy – CoT:          67.22%
Accuracy – CoVe:         67.22%
Accuracy – Self-Refine:  70.00%


### Calculate accuracy per Subject

In [156]:
subject_acc = df_all.groupby("subject")[["correct_cot", "correct_cove", "correct_sr"]].mean().round(3)

display(subject_acc)

,correct_cot,correct_cove,correct_sr
subject,,,
Ciências Humanas e suas Tecnologias,0.978,0.911,0.933
Ciências da Natureza e suas Tecnologias,0.533,0.556,0.556
"Linguagens, Códigos e suas Tecnologias",0.756,0.844,0.800
Matemática e suas Tecnologias,0.422,0.378,0.511


## Analyse Regressions and Improvements

### CoVe

#### Define regressions and improvements

In [184]:
df_cove["regression"] = (
    (df_cove["initial_answer"] == df_cove["ground_truth"]) &
    (df_cove["predicted_cove"] != df_cove["ground_truth"])
)

df_cove["improvement"] = (
    (df_cove["initial_answer"] != df_cove["ground_truth"]) &
    (df_cove["predicted_cove"] == df_cove["ground_truth"])
)

df_cove["changed_initial_answer"] = (
    df_cove["initial_answer"] != df_cove["predicted_cove"]
)

df_cove.head()

,id,question,ground_truth,predicted_cove,correct,baseline_answer,initial_answer,final_answer,verification_qs,verification_a,subject,regression,improvement,changed_initial_answer
0,questao_01,"## Holy War\nOh, so we can hate each other and...",A,A,True,"A expressão ""instead of"" introduz a ideia de s...",A,"A expressão ""instead of"" é utilizada na letra ...",['1. Qual a mensagem principal da letra da can...,[('1. Qual a mensagem principal da letra da ca...,"Linguagens, Códigos e suas Tecnologias",False,False,False
1,questao_02,[[placeholder]]\nO texto estabelece uma relaçã...,A,D,False,"Para responder a essa pergunta, primeiro devem...",D,"Com base nas verificações realizadas, podemos ...",['1. Qual a relação estabelecida no texto entr...,[('1. Qual a relação estabelecida no texto ent...,"Linguagens, Códigos e suas Tecnologias",False,False,False
2,questao_03,[[placeholder]]\nA relação entre as citações a...,C,C,True,"Para responder a essa pergunta, primeiro vamos...",C,Após analisar as citações atribuídas a Albert ...,['1. Quais são as citações atribuídas a Albert...,[('1. Quais são as citações atribuídas a Alber...,"Linguagens, Códigos e suas Tecnologias",False,False,False
3,questao_04,[[placeholder]]A carta da editora Stephanie Al...,E,D,False,"Para responder a essa pergunta, devemos analis...",D,"Com base nas verificações realizadas, a carta ...",['1) Qual é o propósito da carta da editora St...,[('1) Qual é o propósito da carta da editora S...,"Linguagens, Códigos e suas Tecnologias",False,False,False
4,questao_05,## Borderlands/La frontera: the Nem Mestiza\nG...,A,A,True,O texto aborda o preconceito e discriminação e...,A,O texto aborda a pressão e discriminação enfre...,['1. Como os imigrantes residentes nos Estados...,[('1. Como os imigrantes residentes nos Estado...,"Linguagens, Códigos e suas Tecnologias",False,False,False


#### Calculate total regressions (first correct then wrong) and improvements (first wrong then correct)

In [158]:
n_total_cove = len(df_cove)
n_regress_cove = df_cove["regression"].sum()
n_improve_cove = df_cove["improvement"].sum()

print(f"Total: {n_total_cove}")
print(f"Regressions: {n_regress_cove} ({n_regress_cove / n_total_cove:.2%})")
print(f"Improvements: {n_improve_cove} ({n_improve_cove / n_total_cove:.2%})")


Total: 180
Regressions: 4 (2.22%)
Improvements: 0 (0.00%)


#### 📊 CoVe – Regressions and Improvements per Subject

In [186]:
subject_summary_cove = df_cove.groupby("subject").agg(
    total_questions=("id", "count"),
    changes=("changed_initial_answer", "sum"),
    regressions=("regression", "sum"),
    improvements=("improvement", "sum")
).reset_index()

display(subject_summary_cove)


,subject,total_questions,changes,regressions,improvements
0,Ciências Humanas e suas Tecnologias,45,0,0,0
1,Ciências da Natureza e suas Tecnologias,45,3,1,0
2,"Linguagens, Códigos e suas Tecnologias",45,1,0,0
3,Matemática e suas Tecnologias,45,11,3,0


### Self-Refine

#### Extract and normalize answers

In [179]:
df_self_refine["initial_answer"] = df_self_refine["answer_sequence"].apply(
    lambda seq: seq[0] if seq else None
)

df_self_refine["answer_sequence_normalized"] = df_self_refine["answer_sequence"].apply(
    lambda seq: [str(a).strip().upper() for a in seq]
)

df_self_refine["ground_truth_normalized"] = df_self_refine["ground_truth"].str.strip().str.upper()
df_self_refine["predicted_normalized"] = df_self_refine["predicted_sr"].str.strip().str.upper()


#### Set regression and improvement flags

In [178]:
df_self_refine["regression"] = (
    df_self_refine.apply(
        lambda row: row["ground_truth_normalized"] in row["answer_sequence_normalized"]
        and row["predicted_normalized"] != row["ground_truth_normalized"],
        axis=1
    )
)

df_self_refine["improvement"] = (
    (df_self_refine["initial_answer"].str.strip().str.upper() != df_self_refine["ground_truth_normalized"]) &
    (df_self_refine["predicted_normalized"] == df_self_refine["ground_truth_normalized"])
)

df_self_refine.head()

,id,question,ground_truth,predicted_sr,correct,baseline_answer,final_answer,answer_sequence,trace,subject,...,regression,improvement,num_iterations,num_changes,changed_initial_answer,answer_sequence_normalized,ground_truth_normalized,predicted_normalized,changed_and_improved,changed_and_regressed
0,questao_01,"## Holy War\nOh, so we can hate each other and...",A,A,True,"A expressão ""instead of"" é um marcador que int...","A expressão ""instead of"" é um marcador que int...",[A],"[('A expressão ""instead of"" é um marcador que ...","Linguagens, Códigos e suas Tecnologias",...,False,False,1,0,False,[A],A,A,False,False
1,questao_02,[[placeholder]]\nO texto estabelece uma relaçã...,A,D,False,"Para responder a essa pergunta, devemos analis...","Para responder a essa pergunta, devemos analis...",[D],"[('Para responder a essa pergunta, devemos ana...","Linguagens, Códigos e suas Tecnologias",...,False,False,1,0,False,[D],A,D,False,False
2,questao_03,[[placeholder]]\nA relação entre as citações a...,C,C,True,"Para responder a essa pergunta, devemos analis...","Para responder a essa pergunta, devemos analis...",[C],"[('Para responder a essa pergunta, devemos ana...","Linguagens, Códigos e suas Tecnologias",...,False,False,1,0,False,[C],C,C,False,False
3,questao_04,[[placeholder]]A carta da editora Stephanie Al...,E,E,True,"Para responder a essa pergunta, devemos analis...",Analisando a carta da editora Stephanie Allen-...,"[D, E]","[('Para responder a essa pergunta, devemos ana...","Linguagens, Códigos e suas Tecnologias",...,False,True,2,1,True,"[D, E]",E,E,True,False
4,questao_05,## Borderlands/La frontera: the Nem Mestiza\nG...,A,A,True,"Neste texto, Glória Anzaldúa aborda o problema...","Neste texto, Glória Anzaldúa aborda o problema...",[A],"[('Neste texto, Glória Anzaldúa aborda o probl...","Linguagens, Códigos e suas Tecnologias",...,False,False,1,0,False,[A],A,A,False,False


#### Analyse dynamics of answer sequence

##### Calculate number of iterations and changes

In [170]:
df_self_refine["initial_answer"] = df_self_refine["answer_sequence"].apply(lambda seq: seq[0] if seq else None)

df_self_refine["num_iterations"] = df_self_refine["answer_sequence"].apply(len)

df_self_refine["num_changes"] = df_self_refine["answer_sequence"].apply(
    lambda seq: sum(seq[i] != seq[i-1] for i in range(1, len(seq)))
)
df_self_refine["changed_initial_answer"] = (
    df_self_refine["initial_answer"] != df_self_refine["predicted_sr"]
)

df_self_refine

,id,question,ground_truth,predicted_sr,correct,baseline_answer,final_answer,answer_sequence,trace,subject,initial_answer,regression,improvement,num_iterations,num_changes,changed_initial_answer,answer_sequence_normalized,ground_truth_normalized,predicted_normalized
0,questao_01,"## Holy War\nOh, so we can hate each other and...",A,A,True,"A expressão ""instead of"" é um marcador que int...","A expressão ""instead of"" é um marcador que int...",[A],"[('A expressão ""instead of"" é um marcador que ...","Linguagens, Códigos e suas Tecnologias",A,False,False,1,0,False,[A],A,A
1,questao_02,[[placeholder]]\nO texto estabelece uma relaçã...,A,D,False,"Para responder a essa pergunta, devemos analis...","Para responder a essa pergunta, devemos analis...",[D],"[('Para responder a essa pergunta, devemos ana...","Linguagens, Códigos e suas Tecnologias",D,False,False,1,0,False,[D],A,D
2,questao_03,[[placeholder]]\nA relação entre as citações a...,C,C,True,"Para responder a essa pergunta, devemos analis...","Para responder a essa pergunta, devemos analis...",[C],"[('Para responder a essa pergunta, devemos ana...","Linguagens, Códigos e suas Tecnologias",C,False,False,1,0,False,[C],C,C
3,questao_04,[[placeholder]]A carta da editora Stephanie Al...,E,E,True,"Para responder a essa pergunta, devemos analis...",Analisando a carta da editora Stephanie Allen-...,"[D, E]","[('Para responder a essa pergunta, devemos ana...","Linguagens, Códigos e suas Tecnologias",D,False,True,2,1,True,"[D, E]",E,E
4,questao_05,## Borderlands/La frontera: the Nem Mestiza\nG...,A,A,True,"Neste texto, Glória Anzaldúa aborda o problema...","Neste texto, Glória Anzaldúa aborda o problema...",[A],"[('Neste texto, Glória Anzaldúa aborda o probl...","Linguagens, Códigos e suas Tecnologias",A,False,False,1,0,False,[A],A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,questao_176,"Atualmente, há telefones celulares com telas d...",D,D,True,Para encontrar a diferença de tamanho entre as...,Para encontrar a diferença de tamanho entre as...,[D],[('Para encontrar a diferença de tamanho entre...,Matemática e suas Tecnologias,D,False,False,1,0,False,[D],D,D
176,questao_177,Uma imobiliária iniciou uma campanha de divulg...,A,B,False,Para encontrar o mês mais adequado para o clie...,Para encontrar o mês mais adequado para o clie...,[B],[('Para encontrar o mês mais adequado para o c...,Matemática e suas Tecnologias,B,False,False,1,0,False,[B],A,B
177,questao_178,"Um professor de desenho, apresentou uma peça d...",D,D,True,"Para encontrar a escala utilizada no desenho, ...","Para encontrar a escala utilizada no desenho, ...",[D],[('Para encontrar a escala utilizada no desenh...,Matemática e suas Tecnologias,D,False,False,1,0,False,[D],D,D
178,questao_179,A prefeitura de uma cidade planeja construir t...,C,C,True,Para que os postos de saúde sejam equidistante...,Para que os postos de saúde sejam equidistante...,[C],[('Para que os postos de saúde sejam equidista...,Matemática e suas Tecnologias,C,False,False,1,0,False,[C],C,C


##### Checkout cases where the model changed its initial answer

In [171]:
df_self_refine[df_self_refine["changed_initial_answer"]][
    ["id", "initial_answer", "predicted_sr", "ground_truth", "answer_sequence", "improvement", "regression"]
]

,id,initial_answer,predicted_sr,ground_truth,answer_sequence,improvement,regression
3,questao_04,D,E,E,"[D, E]",True,False
16,questao_17,B,A,B,"[B, E, B, A]",False,True
20,questao_21,D,E,C,"[D, E]",False,False
27,questao_28,E,B,B,"[E, B]",True,False
28,questao_29,D,A,A,"[D, A]",True,False
30,questao_31,D,A,D,"[D, A]",False,True
53,questao_54,D,B,D,"[D, B]",False,True
55,questao_56,E,A,A,"[E, A, E, A]",True,False
77,questao_78,D,A,A,"[D, A]",True,False
88,questao_89,C,E,C,"[C, B, C, E]",False,True


##### Changed and improved, changed and regressed

In [175]:
df_self_refine["changed_and_improved"] = (
    df_self_refine["changed_initial_answer"] & df_self_refine["improvement"]
)

df_self_refine["changed_and_regressed"] = (
    df_self_refine["changed_initial_answer"] & df_self_refine["regression"]
)

#### 📊 Self-Refine – Regressions and Improvements per Subject

In [181]:
subject_summary_sr = df_self_refine.groupby("subject").agg(
    total_questions=("id", "count"),
    changes=("changed_initial_answer", "sum"),
    regressions=("regression", "sum"),
    improvements=("improvement", "sum")
).reset_index()

display(subject_summary_sr)

,subject,total_questions,changes,regressions,improvements
0,Ciências Humanas e suas Tecnologias,45,4,2,2
1,Ciências da Natureza e suas Tecnologias,45,11,6,4
2,"Linguagens, Códigos e suas Tecnologias",45,6,2,3
3,Matemática e suas Tecnologias,45,9,7,1


### Compare CoVe and Self-Refine

#### Did they improve or regress on the same questions?